In [1]:
import cv2 
import numpy as np 

def Color_and_Grad(undistorted, RH, RL, SH, SL, HIGH_THRESHOLD, LOW_THRESHOLD):
    # Step 1: HSL → S channel
    hls = cv2.cvtColor(undistorted, cv2.COLOR_BGR2HLS)
    s_channel = hls[:,:,2] # H , W, COLOR CHANNEL   HLS H=0 L=1 S=2
    s_binary = cv2.inRange(s_channel, SL, SH)
    
    # Step 2: BGR → R channel
    r_channel = undistorted[:,:,2] #R==0 G==1 B==2 
    r_binary = cv2.inRange(r_channel, RL, RH)
    
    # Step 3: Gray + Canny
    gray = cv2.cvtColor(undistorted, cv2.COLOR_BGR2GRAY)
    canny = cv2.Canny(gray, HIGH_THRESHOLD, LOW_THRESHOLD)
    #  low threshold  :below it everything is zero that is black 
    # high threshold :above it every point is 255 or white 
    # everything in between is connected to forms line 

    # Step 4: combine
    sr_combined = cv2.bitwise_or(r_binary, s_binary)  
    # 255 or 0 = 255     
    # 0 or  0=0
    # 255 or  255=255
    # 0 or 255=255
    combined = cv2.bitwise_or(canny,sr_combined)
    return sr_combined





# Input video
cap = cv2.VideoCapture(r"D:\computer vision\drive-download-20251028T233500Z-1-001\challenge (1).mp4")

# Get video properties
fourcc = cv2.VideoWriter_fourcc(*'mp4v')   # Codec
fps = int(cap.get(cv2.CAP_PROP_FPS))
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define VideoWriter
out = cv2.VideoWriter("gray.mp4", fourcc, 25, (width, height), isColor=False)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("video ended or breaked")
        break
    
    IMG = Color_and_Grad(frame, 250, 220, 250, 200, 140,200)
    
    # Write the frame (must match outpu size and type)
    out.write(IMG)

    # Display live (optional)
    cv2.imshow("Canny Output", IMG)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

